In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf

In [4]:
import csv
import numpy as np
import pandas as pd

In [ ]:
raw_dataset = tf.data.TFRecordDataset(["arxiv-all.tf_record"])

In [ ]:
for raw_record in tf.data.TFRecordDataset(["arxiv-all.tf_record"]):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)
    break

In [ ]:
example

features {
  feature {
    key: "abstract_contains_deep"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "abstract_contains_embedding"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "abstract_contains_neural"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "abstract_contains_novel"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "abstract_contains_outperform"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "abstract_contains_state-of-the-art"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "accepted"
    value {
      int64_list {
        value: 1
      }
    }
  }
  feature {
    key: "arxiv"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "avg_length_reference_mention_contexts"
    value {
      float_list {
      

In [ ]:
feature_description = {
    'abstract_contains_deep': tf.io.FixedLenFeature([], tf.int64),
    'abstract_contains_embedding': tf.io.FixedLenFeature([], tf.int64),
    'abstract_contains_neural': tf.io.FixedLenFeature([], tf.int64),
    'abstract_contains_novel': tf.io.FixedLenFeature([], tf.int64),
    'abstract_contains_outperform': tf.io.FixedLenFeature([], tf.int64),
    'abstract_contains_state-of-the-art': tf.io.FixedLenFeature([], tf.int64),
    'accepted': tf.io.FixedLenFeature([], tf.int64),
    'arxiv': tf.io.FixedLenFeature([], tf.int64),
    # 'avg_length_reference_mention_contexts': tf.io.VarLenFeature(tf.float32),
    'avg_sentence_length': tf.io.FixedLenFeature([], tf.float32),
    'contains_appendix': tf.io.FixedLenFeature([], tf.int64),
    'id': tf.io.FixedLenFeature([], tf.int64),
    'many_split': tf.io.FixedLenFeature([], tf.int64),
    "most_recent_reference_year": tf.io.FixedLenFeature([], tf.int64),
    'num_authors': tf.io.FixedLenFeature([], tf.int64),
    'num_recent_references': tf.io.FixedLenFeature([], tf.int64),
    'num_ref_to_equations': tf.io.FixedLenFeature([], tf.int64),
    'num_ref_to_figures': tf.io.FixedLenFeature([], tf.int64),
    'num_ref_to_sections': tf.io.FixedLenFeature([], tf.int64),
    'num_ref_to_tables': tf.io.FixedLenFeature([], tf.int64),
    'num_ref_to_theorems': tf.io.FixedLenFeature([], tf.int64),
    'num_references': tf.io.FixedLenFeature([], tf.int64),
    'num_refmentions': tf.io.FixedLenFeature([], tf.int64),
    'num_sections': tf.io.FixedLenFeature([], tf.int64),
    'num_uniq_words': tf.io.FixedLenFeature([], tf.int64),
    'title_contains_deep': tf.io.FixedLenFeature([], tf.int64),
    'title_contains_embedding': tf.io.FixedLenFeature([], tf.int64),
    'title_contains_gan': tf.io.FixedLenFeature([], tf.int64),
    'title_contains_neural': tf.io.FixedLenFeature([], tf.int64),
    'title_length': tf.io.FixedLenFeature([], tf.int64),
    'token_ids': tf.io.VarLenFeature(tf.int64),
    'token_mask': tf.io.VarLenFeature(tf.int64),
    'venue': tf.io.FixedLenFeature([], tf.int64),
    'year': tf.io.FixedLenFeature([], tf.int64)
}

In [ ]:
feature_keys = feature_description.keys()

In [ ]:
def decode(serialized_example):
    parsed_example = {}
    for key, feature in feature_description.items():
        try:
            # Attempt to parse the feature as defined in feature_description
            parsed_example[key] = tf.io.parse_single_example(
                serialized_example, {key: feature}
            )[key]
        except ValueError:
            # If parsing fails, try parsing as VarLenFeature with different data types
            for dtype in [tf.int64, tf.float32, tf.string]:
                try:
                    parsed_example[key] = tf.io.parse_single_example(
                        serialized_example, {key: tf.io.VarLenFeature(dtype)}
                    )[key]
                    break
                except ValueError:
                    continue
    return parsed_example



In [ ]:
def decode(serialized_example):
    parsed_example = {}
    for key in feature_keys:

        if key == 'avg_length_reference_mention_contexts':
          print("hahaha")
          parsed_example[key] = tf.io.parse_single_example(
              serialized_example,
              {key: tf.io.VarLenFeature(tf.float32)}
          )[key]
        else:

          for dtype in [tf.int64, tf.float32, tf.string]:
              try:
                  parsed_example[key] = tf.io.parse_single_example(
                      serialized_example,
                      {key: tf.io.VarLenFeature(dtype)}
                  )[key]
                  break
              except ValueError:
                  continue
    return parsed_example


In [ ]:
raw_dataset = tf.data.TFRecordDataset("arxiv-all.tf_record")
decoded_dataset = raw_dataset.map(decode)

In [ ]:
with open("PeerRead.csv", mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=feature_description.keys())
    writer.writeheader()

    for record in decoded_dataset:
        record_dict = {}
        for key in feature_description.keys():
            feature = record[key]
            if feature is not None:
                if isinstance(feature, tf.SparseTensor):
                    record_dict[key] = ','.join(map(str, tf.sparse.to_dense(feature).numpy()))
                else:
                    record_dict[key] = feature.numpy()
            else:
                record_dict[key] = None
        writer.writerow(record_dict)


In [ ]:
df = pd.read_csv('PeerRead.csv')

In [ ]:
df.shape

(11778, 33)

In [ ]:
df.head()

,abstract_contains_deep,abstract_contains_embedding,abstract_contains_neural,abstract_contains_novel,abstract_contains_outperform,abstract_contains_state-of-the-art,accepted,arxiv,avg_sentence_length,contains_appendix,...,num_uniq_words,title_contains_deep,title_contains_embedding,title_contains_gan,title_contains_neural,title_length,token_ids,token_mask,venue,year
0,0,0,0,0,0,0,1,0,24.723810,0,...,1991,0,0,0,0,54,"101,2057,8970,1037,7705,2005,22910,2838,2008,1...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",5,2007
1,0,0,0,0,0,0,1,0,28.226257,1,...,2764,0,0,0,0,39,"101,9420,5398,1997,11702,2006,6550,2005,7689,1...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",4,2007
2,0,0,0,0,0,0,1,0,14.513618,1,...,1000,0,0,0,0,48,"101,1999,2023,4087,3189,1045,2265,2008,1996,79...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",5,2007
3,0,0,0,0,0,0,1,0,25.314816,0,...,1303,0,0,0,0,34,"101,2391,8044,2024,4520,1997,2685,1999,2048,20...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",5,2007
4,0,0,0,0,0,0,1,1,25.141975,0,...,1451,0,0,0,0,95,"101,2087,2783,2773,17547,3001,2191,2224,1997,1...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",6,2008


In [ ]:
with open('vocab.txt', 'r') as file:
    txt = file.read()
    tokens = txt.split('\n')

In [ ]:
token_id_to_token = {idx: token for idx, token in enumerate(tokens)}

In [ ]:
abstracts = []
for index, row in df.iterrows():
  token_id = row['token_ids']
  token_ids = [int(id) for id in token_id.split(',') if id.isdigit()]
  tokens = [token_id_to_token.get(id, '') for id in token_ids if id != 0]
  reconstructed = []
  for token in tokens:
    if token == "[CLS]" or token == "[SEP]":
        continue
    if token.startswith("##"):
        if reconstructed:
            reconstructed[-1] += token[2:]
    else:
        reconstructed.append(token)
  abstract = ' '.join(reconstructed)
  abstracts.append(abstract)


In [ ]:
df['abstract'] = abstracts

In [ ]:
df.to_csv('PeerRead_with_abstract.csv')

### TOP K (wrong one)

In [5]:
df = pd.read_csv("/content/drive/MyDrive/PeerRead_with_abstract.csv")

In [6]:
df.head()

,Unnamed: 0,abstract_contains_deep,abstract_contains_embedding,abstract_contains_neural,abstract_contains_novel,abstract_contains_outperform,abstract_contains_state-of-the-art,accepted,arxiv,avg_sentence_length,...,title_contains_deep,title_contains_embedding,title_contains_gan,title_contains_neural,title_length,token_ids,token_mask,venue,year,abstract
0,0,0,0,0,0,0,0,1,0,24.723810,...,0,0,0,0,54,"101,2057,8970,1037,7705,2005,22910,2838,2008,1...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",5,2007,we introduce a framework for filtering feature...
1,1,0,0,0,0,0,0,1,0,28.226257,...,0,0,0,0,39,"101,9420,5398,1997,11702,2006,6550,2005,7689,1...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",4,2007,observations consisting of measurements on rel...
2,2,0,0,0,0,0,0,1,0,14.513618,...,0,0,0,0,48,"101,1999,2023,4087,3189,1045,2265,2008,1996,79...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",5,2007,in this technical report i show that the brier...
3,3,0,0,0,0,0,0,1,0,25.314816,...,0,0,0,0,34,"101,2391,8044,2024,4520,1997,2685,1999,2048,20...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",5,2007,point clouds are sets of points in two or thre...
4,4,0,0,0,0,0,0,1,1,25.141975,...,0,0,0,0,95,"101,2087,2783,2773,17547,3001,2191,2224,1997,1...","1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...",6,2008,most current word prediction systems make use ...


In [7]:
df.shape

(11778, 35)

In [8]:
original_buzzy = ['deep', 'neural', 'embed', 'adversarial net']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def find_similar_words(df, vectorizer):
    X = vectorizer.fit_transform(df['abstract'])
    expanded_buzzy = set()
    for word in original_buzzy:
        if word in vectorizer.get_feature_names_out():
            word_vec = vectorizer.transform([word]).toarray()
            cos_sim = cosine_similarity(word_vec, X).flatten()
            similar_indices = np.argwhere(cos_sim > 0.3).flatten()
            similar_words = set(np.array(vectorizer.get_feature_names_out())[similar_indices])
            expanded_buzzy.update(similar_words)

    return expanded_buzzy

In [ ]:
# tfidf gives more importance to the less frequent
tfidf_vectorizer = TfidfVectorizer()
expand_buzzy_tfidf = find_similar_words(df, tfidf_vectorizer)

In [ ]:
expand_buzzy_tfidf

{'bm',
 'cntf',
 'converse',
 'cybersecurity',
 'daml',
 'declined',
 'degraded',
 'demarcated',
 'distinctive',
 'elaborates',
 'enforces',
 'eodule',
 'estate',
 'fabrics',
 'furhter',
 'generalizability'}

In [ ]:
# count treats all words equally focusing on the frequencies
count_vectorizer = CountVectorizer()
expand_buzzy_count = find_similar_words(df, count_vectorizer)

In [ ]:
expand_buzzy_count

{'auml',
 'aurora4',
 'bm',
 'boreholes',
 'bttp',
 'byproducts',
 'chorales',
 'cntf',
 'cntk',
 'communicating',
 'complementarity',
 'conducted',
 'consult',
 'converges',
 'converse',
 'cornerstone',
 'corrupt',
 'cosmic',
 'cybersecurity',
 'daml',
 'declined',
 'degraded',
 'demarcated',
 'distinctive',
 'edible',
 'eigenvector',
 'elaborates',
 'enforces',
 'eodule',
 'epistemic',
 'estate',
 'extensively',
 'extubation',
 'fabrics',
 'ferns',
 'flatness',
 'forgoing',
 'formation',
 'furhter',
 'generalizability',
 'genericity',
 'hacker'}

In [ ]:
# probably including both

for w in expand_buzzy_tfidf:
  if w in expand_buzzy_count:
    print(w)

degraded
cntf
demarcated
estate
eodule
daml
elaborates
declined
cybersecurity
enforces
fabrics
furhter
generalizability
distinctive
converse
bm


### Bert Embedding

In [ ]:
# pip install transformers

In [9]:
from transformers import BertTokenizer, BertModel
import torch

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [11]:
def tokenize(text):
  marked_text = "[CLS] " + text + " [SEP]"
  tokens = tokenizer.tokenize(marked_text)
  return tokens

In [13]:
  def get_embedding(text):
    tokens = tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
    segments_ids = [1] * len(indexed_tokens)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    model.eval()

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = hidden_states[-2][0]
    sentence_embedding = torch.mean(token_embeddings, dim=0)
    return sentence_embedding

In [14]:
from scipy.spatial.distance import cosine

In [15]:
original_buzzy_embeddings = {}
for w in original_buzzy:
  original_buzzy_embeddings[w] = get_embedding(w)

In [16]:
df['tokens'] = df['abstract'].apply(tokenize)

In [17]:
def sim(tokens, original_buzzy_embeddings, threshold=0.7):
  similar_words = []
  print("start")
  for token in tokens:
    token_embedding = get_embedding(token)
    for word, emb in original_buzzy_embeddings.items():
      similarity = 1 - cosine(token_embedding, emb)
      if similarity > threshold:
        similar_words.append(word)
  print("done")
  return similar_words

In [18]:
results = set(df['tokens'].apply(lambda x: sim(x, original_buzzy_embeddings)))

start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done
start
done

KeyboardInterrupt: ignored

In [ ]:
results

In [ ]:
# both not in it
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2))
print('adversarial net' in tfidf_vectorizer.get_feature_names_out())

NotFittedError: ignored